# Week 2 Optimisation

## Strategy

Stats we will be taking into account in our objective functions we are trying to maximise within the constraints:

- m1. Average Fantasy Points in 2019 season (weight: 30%)
- m2. Points from the last race (45%)
- m3. Qualified in Top 10 (25%)

In [1]:
import requests
import pandas as pd
from pulp import *
import re

## Get Data

In [2]:
r = requests.get('https://fantasy-api.formula1.com/partner_games/f1/players')

In [3]:
cost = pd.DataFrame(r.json()['players'] )
cost = cost.set_index('id').reset_index(drop=True)
cost.head(2)

,first_name,last_name,team_name,position,position_id,position_abbreviation,price,current_price_change_info,status,injured,...,driver_data,constructor_data,born_at,season_prices,num_fixtures_in_gameweek,has_fixture,display_name,external_id,profile_image,misc_image
0,Scuderia Ferrari Mission Winnow,,Scuderia Ferrari Mission Winnow,Constructor,2,CR,26.7,"{'current_selection_count': 61840, 'current_se...",None,False,...,None,"{'best_finish': 1, 'best_finish_count': 239, '...",None,"[{'game_period_id': 1, 'price': 27.4}, {'game_...",2,True,Ferrari,526:59:FER,{'url': 'https://f1imageuploads.s3.amazonaws.c...,{'url': None}
1,Williams Racing,,Williams Racing,Constructor,2,CR,6.5,"{'current_selection_count': 9606, 'current_sel...",None,False,...,None,"{'best_finish': 1, 'best_finish_count': 114, '...",None,"[{'game_period_id': 1, 'price': 6.5}, {'game_p...",2,True,Williams,481:172:WIL,{'url': 'https://f1imageuploads.s3.amazonaws.c...,{'url': None}


In [4]:
cost.columns

Index(['first_name', 'last_name', 'team_name', 'position', 'position_id',
       'position_abbreviation', 'price', 'current_price_change_info', 'status',
       'injured', 'injury_type', 'banned', 'ban_type',
       'streak_events_progress', 'chance_of_playing', 'picked_percentage',
       'team_abbreviation', 'team_id', 'headshot', 'known_name',
       'jersey_image', 'score', 'humanize_status', 'shirt_number', 'country',
       'is_constructor', 'season_score', 'driver_data', 'constructor_data',
       'born_at', 'season_prices', 'num_fixtures_in_gameweek', 'has_fixture',
       'display_name', 'external_id', 'profile_image', 'misc_image'],
      dtype='object')

In [5]:
cost['q_top_10'] = cost['streak_events_progress'].apply(lambda x: x['top_ten_in_a_row_qualifying_progress'])

In [17]:
cost['q_top_10'] = cost['q_top_10'].astype('float')

Get 2019 data and merge with latest data to get final data

In [6]:
driver = pd.read_pickle('../2019_Driver_Points.pickle')
constructor = pd.read_pickle('../2019_Constructor_Points.pickle')

In [7]:
cost.loc[cost.position=='Driver', 'display_name']=cost[cost.position=='Driver']['display_name'].str.replace(' ', '')

In [18]:
driver_data = cost[['display_name','position','price', 'season_score', 'team_abbreviation', 'q_top_10']][cost.position=='Driver'].merge(
    driver[['Name', 'AvgPoints']],
    left_on = 'display_name',
    right_on = 'Name',
    how='left'
).drop(columns=['Name'])
driver_data['AvgPoints'] = driver_data.AvgPoints.fillna(0)

In [19]:
constructor_data = cost[['display_name', 'position','team_abbreviation', 'price', 'season_score', 'q_top_10']][cost.position=='Constructor'].merge(
    constructor[['Constructor','team_abbreviation', 'AverageCon']],
    on='team_abbreviation',
    how='left'
).drop(columns=['Constructor']).rename(
    columns={
        'AverageCon': 'AvgPoints',
    }
)
constructor_data['AvgPoints'] = constructor_data.AvgPoints.fillna(0)

In [20]:
data = pd.concat(
    [constructor_data, driver_data]
).drop(columns=['team_abbreviation']).reset_index(drop=True)

In [21]:
data.head(2)

,display_name,position,price,season_score,q_top_10,AvgPoints
0,Ferrari,Constructor,26.7,48.0,0.0,45.8
1,Williams,Constructor,6.5,15.0,0.0,11.1


standardise variables

In [22]:
def z_score(series):
    mean = series.mean()
    std = series.std()
    
    return (series - mean) / std

In [23]:
data['season_score'] = z_score(data['season_score'])
data['q_top_10'] = z_score(data['q_top_10'])
data['AvgPoints']  = z_score(data['AvgPoints'])

## Optimisation

Create decision variables

In [24]:
constructor_var = LpVariable.dict(
    'Constructor', 
    data['display_name'][data.position=='Constructor'], 
    0, 
    cat='Binary'
)
driver_var = LpVariable.dict(
    'Driver', 
    data['display_name'][data.position=='Driver'], 
    0, 
    cat='Binary'
)

In [25]:
current_team = constructor_var['Mercedes'] + driver_var['L.Hamilton'] + driver_var['K.Raikkonen'] + driver_var['S.Perez'] + driver_var['G.Russell'] + driver_var['D.Kvyat']

Define the problem

In [26]:
prob = LpProblem('Fantasty_F1_Week_2', LpMaximize)

# Objective function
driver_rewards = []
constructor_rewards = []

driver_coef = list(
    data[data.position=='Driver'].eval(
        '0.3*AvgPoints + 0.25*q_top_10 + 0.2*season_score'
    )
)
constructor_coef = list(
    data[data.position=='Constructor'].eval(
        '0.3*AvgPoints + 0.25*q_top_10 + 0.2*season_score'
    )
)
driver_rewards += lpSum( 
    [a*b for a, b in zip(driver_coef, driver_var.values()) ]
)
constructor_rewards += lpSum( 
    [a*b for a, b in zip(constructor_coef, constructor_var.values()) ]
)

prob += driver_rewards + constructor_rewards, 'Total Points Scored'

# Constraints
prob += lpSum(driver_var) == 5, 'Max Number of Drivers'
prob += lpSum(constructor_var) == 1, 'Max Number of Constructor'
prob += current_team == 4, 'Max 2 changes'

driver_costs = []
constructor_costs = []
driver_price = list(data[data.position=='Driver'].price)
constructor_price = list(data[data.position=='Constructor'].price)

driver_costs += lpSum(
    [a*b for a,b in zip(driver_price, driver_var.values())]
)
constructor_costs += lpSum(
    [a*b for a,b in zip(constructor_price, constructor_var.values())]
)

prob += driver_costs + constructor_costs <= 100, 'Total Cost of Team'

In [27]:
prob.solve()

1

In [28]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [29]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [30]:
summary(prob)

Variables:

Constructor_Mercedes = 1.0
Driver_A.Giovinazzi = 1.0
Driver_G.Russell = 1.0
Driver_L.Hamilton = 1.0
Driver_L.Norris = 1.0
Driver_S.Perez = 1.0
---------------------------------------

Constraints:
32.2*1.0 + 8.1*1.0 + 5.9*1.0 + 31.2*1.0 + 12.5*1.0 + 9.7*1.0 = 99.60000000000001
---------------------------------------

Score:
1.8681219723912368*1.0 + 0.49058231028966415*1.0 + 0.6769380979792454*1.0 + 0.9501987631835171*1.0 + 0.2544055815254332*1.0 + 0.2319732783835322*1.0 = 1.6732326304477454


# END